# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes the point $P = dG = (P_x, P_y)$: his public key is the $x$-coordinate, $P_x$

In [146]:
def hashPDF(file, BLOCK_SIZE):
    # hash=sha256()
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(BLOCK_SIZE) # Read from the file. Take in the amount declared above
        M = sl.sha256(fb)
    return M
file = "D:/Documentos/pdf-test.pdf"
size = os.path.getsize(file) 

In [117]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

[{'nameUser': '0',
  'privateKey': 'd222bae3d092d4070b910d7cce4ee4cad5d581ccaea8f040bf7e029c1c6e01d8',
  'publicKey': '0a2466fa12ff6b8bfd5befd8e6b91dd9f10da6bc592964474bb9854da9a22b92'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, the $x$-coordinate of $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A computes $e = h(R || P || M)$
- A computes $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [147]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =", user[0]["publicKey"])
print("Signature =",sig.hex())

PublicKey = 822e75cd09a8a8c033d73b9c509ddc2c05329b3777fbfb9e81dc082c20e75a89
Signature = fa8b91ec913d512c9e253aa167b7a7783f0526dae4ce3aaccc9c7901f2090501ef23bd05303b21b2f484ae7ead4e86de3486b1147ac5b09c12d9fa1315bf9aa1


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user $i$ computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(R || \tilde{X}|| M)$
- Every user computes $s_i = r_i + cd_i a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [34]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

print('The private keys are: ')
print('--------------------------')
for u in users:
    pk = u['privateKey']
    print('Usuario ' + u['nameUser'] + ':', pk)

Aggregated key = 142cf6d8a0fcea2900caa8d116cfe08e517c8565763486abf7828e0a4dfed669
Signature = b0614dfe268d84b465d08e56067bfed8fd29915f45e913c0da037974e5527a8a12d01a27734feb98626044eb9d72917a37ce71f38acd3b33817d1e019b64db41
The private keys are: 
--------------------------
Usuario 0: 1524e9f04aaf6c8b764c08bdf3775248ed959184f328104cf0375081dca1ba85
Usuario 1: 32cd7cf2d876d63a259f00ba4a6405a78a305e33d095209b0c861885482ee9a1


In [28]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig_sign_test(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

KeyboardInterrupt: Interrupted by user

## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(R_1||...||R_{\nu}|| \tilde{X} || M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(R || \tilde{X}||M)$, then every user $i$ computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [31]:
import create_keypair as ckp
import schnorr_lib as sl
import os
# from hashlib import sha256

n_keys = int(input("Insert the number of keys to be generated:"))
# n_keys = 2
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = 3dd9f89da17633fa5be72986a605afdc3477a2663bd0661198d59a93d0fb3f62
Signature = ca50387b9e098aac7f7396551091f834c0a0c760e40b5fcedb598a3de2fc4136e871d779bfc01300e8806f26779628070fccf65b23fd58595f7deb11f89d6d00


## Verify Schnorr

- $B$, the receiver, computes the point $sG$
- $B$ computes $e = h(R || P || M)$ $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$
- if $sG = R + eP$, the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$

In [4]:
import schnorr_lib as sl

# M = input("Insert the message to verify:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key


In [88]:
import schnorr_lib as sl

a = sl.pubkey_gen_from_hex('59e9e3efc71935c1c89cb4d8649cb1854fd6925e9de7cf53983def4720ea62a4')
b = sl.pubkey_gen_from_hex('d87112722e49137dd7af0546b1522d05804dae37b2ab59151a836d34ad3ac2e9')

print(a)
print(b)

b'\xba\xd4\x90\x1e\x88\x80\xd2VL\xf8w\xd0\x7f\xbe\xc4\xf0\xfbu\xa8\x80\xcd\xa8\xdc\t.\x9d\xd4\xceB \xd7\x18'
b"\x99\x15'HY\x96\xe8\xff\x9f\xef\x895:\xcb\xba\xd7\r\x1d\xe5.\xb0\xec`\xaf%\xd8\x87Q~\x80\xd8."


In [12]:
a+b

b"\xc5\xf9\x15\x16M\xfd\x96U8r=c\x9a\x91\xf3\x8e\x9e\x8b\xcc\x1c!\x16\x96\x00\x17O\xbfA0\x0b]\xb6\x99\x15'HY\x96\xe8\xff\x9f\xef\x895:\xcb\xba\xd7\r\x1d\xe5.\xb0\xec`\xaf%\xd8\x87Q~\x80\xd8."

In [89]:
di = sl.int_from_hex('59e9e3efc71935c1c89cb4d8649cb1854fd6925e9de7cf53983def4720ea62a4')

Pi = sl.pubkey_point_gen_from_int(di)

l = sl.bytes_from_point(Pi)
print(l)

b'\xba\xd4\x90\x1e\x88\x80\xd2VL\xf8w\xd0\x7f\xbe\xc4\xf0\xfbu\xa8\x80\xcd\xa8\xdc\t.\x9d\xd4\xceB \xd7\x18'


In [68]:
with open('claves_esquema.txt', 'r+') as f:
    for line in f.readlines():
        
        print(str.encode(line))
        

b'\\xd2\\xd7\\xe9\\xf06}\\xae\\xb9\\xdbN\\xdcl\\x99I\\x90&\\xb9\\x94\\xad\\xf7\\xcb\\xe5\\n\\xfd\\xe3=D\\xd9\\xb0\\x9eq\\xec\n'
b'\\xca\\xccmg.\\xd1\\xc4\\x92M\\xc7O\\x8f|\\xd0n\\xb1hy\\xe9\\xbb2k\\x08\\x15p\\xa04\\xbd\\x9f\\xa7\\xbe\\xcb\n'
b'8\\n\\xc9\\xb4\\xf0\\xd5p\\xec\\x15\\xe5\\x10|]\\x95\\x0c\\xcd\\x03Gk:\\xd7?[\\x0ccus\\x9aT6\\x0ca\n'
b'*k\\x0b\\xaa\\xeb\\x8c\\x8dJ\\x12\\xcayA\\x07]v\\x97\\xd9\\xc7\\xf6\\xf4\\xb2\\xf6\\x0b\\xb8(\\xa9W=\\xd9x\\xdd\\x98\n'


In [141]:
import json
import ast
with open('claves.json') as json_file:
    data = json.load(json_file)

for i in data:
    print(i['Ri'])

mh = ast.literal_eval(data[0]['Ri'])
type(mh)

(24580980305957555103045113148648703534438321682711720736306625853305621270336, 20713868159575387609769455025290222095543108921563027241775336249791269973015)
(93252495296415218193867072540494209341928978276519761996748888892937766185582, 70278387588714629666492194513949237952114789670643427583012087451060104410297)


tuple

In [182]:
import json

n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
with open('claves.json') as json_file:
    data = json.load(json_file)

L = b''
for i in data: 
    int_clave = int(i['Clave Publica'])
    byte_clave = sl.bytes_from_int(int_clave)
    L += byte_clave

L = sl.sha256(L)

Rsum = None
X = None

lista_ai = []
for u in data:
    int_clave = int(u['Clave Publica'])
    byte_clave = sl.bytes_from_int(int_clave)
    int_ri = ast.literal_eval(u['Ri'])

    ai = sl.int_from_bytes(sl.sha256(L + byte_clave))
    lista_ai.append(ai)

    X = sl.point_add(X, sl.point_mul(byte_clave, ai))

    Rsum = sl.point_add(Rsum, int_ri)   

c = sl.int_from_bytes(sl.tagged_hash("BIP0340/challenge", (sl.bytes_from_point(Rsum) + sl.bytes_from_point(X) + M))) % n

with open('users.json', 'r') as f:
    data_1 = json.load(f)

priv_key = data_1['users'][0]['privateKey']
priv_key = sl.int_from_hex(priv_key)

ki = ast.literal_eval(data[0]['Ri'])
ki = sl.bytes_from_point(ki)
ki = sl.int_from_bytes(ki)


s = (c * priv_key* lista_ai[0]+ ki) % n




In [165]:
c

68331271218444668060378632121902164747310612146524190360195753234852345666484

In [171]:
lista_ai[0]

46643961751578825509459818638095196334912193712145300047638678117453044880802

In [179]:
o = ast.literal_eval(data[0]['Ri'])
u = sl.bytes_from_point(o)
p = sl.int_from_bytes(u)
p

24580980305957555103045113148648703534438321682711720736306625853305621270336

In [172]:
priv_key

78628772642100123313386464286175402563176252166562872321852210455076943245811

In [175]:
s

250608863760748975628293848266937422410037547571723902197668827754895431514082472823800886070806442055447656902776499080418505196986514110273078358891601507361331425224231854618633752141305576756645541960247012591633507140708136248

In [181]:
s

178702237320023093114683832571444789129456612099300630326332035887291339658031831142496675963187830709847320927838817329439970615657879531322978917904433996145710504122191803567321582484523556166689129565908787700669342786208359704

In [183]:
p = [{"usario": 0, "firma": 30653941572392250832711009887025900278911259557207665859295281793781551910293, "firma agregada": [83594726559429963489371960898066441361664468358327113063543463655156385472359, 52305255534015268575568636279548296746875170664080141202572987595110032487318]},
{"usario": 1, "firma": 95934454112207769408222792948849277920240126181238466246975522674184260563858, "firma agregada": [83594726559429963489371960898066441361664468358327113063543463655156385472359, 52305255534015268575568636279548296746875170664080141202572987595110032487318]}]

In [188]:
for i in p:
    print(i['firma'])

30653941572392250832711009887025900278911259557207665859295281793781551910293
95934454112207769408222792948849277920240126181238466246975522674184260563858


In [196]:
with open('data.json') as json_file:
    data = json.load(json_file)

clave_p = data['Firma']

print(len(clave_p))
#sig = sl.bytes_from_point(clave_p)

#clave_p = clave_p.hex()

clave_agregada = data['Firma Agregada']

print(len(clave_agregada))

#pubkey = sl.bytes_from_int(clave_agregada)



    
#if len(pubkey) != 32:
#    raise ValueError('The public key must be a 32-byte array.')
#if len(sig) != 64:
#    raise ValueError('The signature must be a 64-byte array.')
#P = lift_x_even_y(pubkey)
#r = get_int_R_from_sig(sig)
#s = get_int_s_from_sig(sig)
#if (P is None) or (r >= p) or (s >= n):
#    False
#e = int_from_bytes(tagged_hash("BIP0340/challenge", get_bytes_R_from_sig(sig) + pubkey + msg)) % n
#R = point_add(point_mul(G, s), point_mul(P, n - e))
#if (R is None) or (not has_even_y(R)):
        # print("Please, recompute the sign. R is None or has even y")
#    False
#if x(R) != r:
        # print("There's something wrong")
#    False
#True

155
77
